In [1]:
import os


In [2]:
%pwd

'e:\\image_recog_project\\potato-disease-class\\research'

In [3]:
os.chdir("../")

In [4]:

%pwd

'e:\\image_recog_project\\potato-disease-class'

In [5]:
#entity is nothing but the return type of the function


In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class dataingestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [28]:
from potClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from potClassifier.utils.common import read_yaml,create_directories


In [20]:
read_yaml

<function read_yaml at 0x00000202C86DBE20>

In [21]:
#config in src

In [29]:
class configurationmanager:
    def __init__(self, 
                 config_filepath: CONFIG_FILE_PATH,
                 params_filepath: PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->dataingestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_inges_config = dataingestionconfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )
        return data_inges_config



In [23]:
#update components


In [24]:
import os
import urllib.request as request
import zipfile
from potClassifier import logger
from potClassifier.utils.common import get_size

In [30]:
class DataIngestion:
    def __init__(self,config : dataingestionconfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(url=self.config.source_URL,filename =self.config.local_data_file)
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists. Skipping download.")
    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir 

        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(path=unzip_path)
           

In [32]:
try:
    config = configurationmanager(config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2025-02-18 18:49:20,075: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-18 18:49:20,077: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-18 18:49:20,080: INFO: common: created directory at: artifacts]
[2025-02-18 18:49:20,084: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-18 18:50:59,322: INFO: 3401366041: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 23138006
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9a0aa8af0dcec8536bf9f19b1e5f63f99b2087ba11fc5cccac8ab3c1a83991ba"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 98D8:2ED9B8:3F4729:5BC0A4:67B48692
Accept-Ranges: bytes
Date: Tue, 18 Feb 2025 13:19:20 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100031-HYD
X-Cache: